<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
import sys
sys.path.append('..')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [2]:
import taiko as tk
from taiko.play import *
from taiko.performance import *
from taiko.model import *
from taiko.tools.database import *
from taiko.tools.config import *

from sklearn.model_selection import train_test_split
import lightgbm as lgb
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import posixpath
from skimage.io import imshow, imsave, imread

from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

sns.set(font_scale=1.5)

Using TensorFlow backend.


In [6]:
record_df = load_record_df()
record_df = record_df[(record_df['song_id'] >= 1) & (record_df['song_id'] <= 4)]
record_df

,drummer_name,song_id,left_sensor_datetime,right_sensor_datetime,capture_datetime
1,aaaaa,1,L_2018-09-28_111304.csv,R_2018-09-28_111305.csv,capture_2018_09_28_11_13_05
2,aaaaa,2,L_2018-09-28_111610.csv,R_2018-09-28_111611.csv,capture_2018_09_28_11_16_11
3,aaaaa,3,L_2018-09-28_111936.csv,R_2018-09-28_111937.csv,capture_2018_09_28_11_19_37
4,aaaaa,4,L_2018-09-28_112353.csv,R_2018-09-28_112355.csv,capture_2018_09_28_11_23_55
5,aaaaa,1,L_2018-09-28_112912.csv,R_2018-09-28_112913.csv,capture_2018_09_28_11_29_13
6,aaaaa,2,L_2018-09-28_113242.csv,R_2018-09-28_113244.csv,capture_2018_09_28_11_32_44
7,aaaaa,3,L_2018-09-28_113531.csv,R_2018-09-28_113532.csv,capture_2018_09_28_11_35_32
8,aaaaa,4,L_2018-09-28_113909.csv,R_2018-09-28_113910.csv,capture_2018_09_28_11_39_10
9,aaaaa,1,L_2018-09-28_114340.csv,R_2018-09-28_114340.csv,capture_2018_09_28_11_43_41
10,aaaaa,2,L_2018-09-28_114710.csv,R_2018-09-28_114713.csv,capture_2018_09_28_11_47_13


In [7]:
lgbm = LGBM()

100%|██████████| 12/12 [00:00<00:00, 79.49it/s]

aaaaa
carolyn
cuxi
fatfat
eve
oliver
chris
sheep
howeverover
celiven
john
kdchang


In [35]:
pfs = {}
for id_, row in record_df.iterrows():
    try:
        pf = get_performance(id_=id_)
        x = pf.drop('timestamp', axis=1)
        ts = pf['timestamp']
        pred_df = pd.DataFrame(data={
            'timestamp': ts,
            'hit_type': lgbm.predict(x)
        })
        pfs[id_] = pred_df
    except Exception as e:
        pass

In [26]:
len(pfs)

79

In [67]:
tmp_mat = []
for i_ in pfs.keys():
    for j_ in pfs.keys():
        if i_ >= j_:
            continue
        a_df = pfs[i_]
        b_df = pfs[j_]
        
        lst = [i_, j_]
        for label in range(6):
            x = np.array(a_df[a_df['hit_type'] == label].timestamp)
            if(len(x) == 0):
                x = np.array([0])
            x -= x[0]
            y = np.array(b_df[b_df['hit_type'] == label].timestamp)
            if(len(y) == 0):
                y = np.array([0])
            y -= y[0]
            distance, _ = fastdtw(x, y, dist=euclidean)
            lst += [distance]
        tmp_mat.append(lst)

final_df = pd.DataFrame(data=tmp_mat)

[[1, 5, 202.19930005073547, 87.20011639595032, 273.59890389442444, 621.999703168869, 441.80000472068787, 12.200003862380981]]
[[1, 5, 202.19930005073547, 87.20011639595032, 273.59890389442444, 621.999703168869, 441.80000472068787, 12.200003862380981], [1, 9, 152.09990167617798, 87.59991598129272, 127.90000557899475, 58.60000157356262, 501.49960017204285, 8.700003147125244]]
[[1, 5, 202.19930005073547, 87.20011639595032, 273.59890389442444, 621.999703168869, 441.80000472068787, 12.200003862380981], [1, 9, 152.09990167617798, 87.59991598129272, 127.90000557899475, 58.60000157356262, 501.49960017204285, 8.700003147125244], [1, 13, 138.5999050140381, 236.29990696907043, 185.59960460662842, 757.4998016357422, 309.5000002384186, 4.100002765655518]]
[[1, 5, 202.19930005073547, 87.20011639595032, 273.59890389442444, 621.999703168869, 441.80000472068787, 12.200003862380981], [1, 9, 152.09990167617798, 87.59991598129272, 127.90000557899475, 58.60000157356262, 501.49960017204285, 8.70000314712524

[[1, 5, 202.19930005073547, 87.20011639595032, 273.59890389442444, 621.999703168869, 441.80000472068787, 12.200003862380981], [1, 9, 152.09990167617798, 87.59991598129272, 127.90000557899475, 58.60000157356262, 501.49960017204285, 8.700003147125244], [1, 13, 138.5999050140381, 236.29990696907043, 185.59960460662842, 757.4998016357422, 309.5000002384186, 4.100002765655518], [1, 25, 644.8999946117401, 273.5995993614197, 1058.2942011356354, 166.40000748634338, 639.7990009784698, 7.900001764297485], [1, 29, 206.69990396499634, 143.19991064071655, 135.29989385604858, 706.8995971679688, 139.99939966201782, 22.000003814697266], [1, 33, 708.6999976634979, 244.59979820251465, 95.19990062713623, 79.80000162124634, 216.899400472641, 23.20000195503235], [1, 37, 113.69999957084656, 539.099202632904, 231.1997001171112, 5177.492004394531, 72.60000109672546, 30.40001130104065], [1, 42, 216.89959478378296, 109.80009436607361, 212.89979791641235, 550.4985010623932, 308.4997992515564, 4.2000017166137695]

[[1, 5, 202.19930005073547, 87.20011639595032, 273.59890389442444, 621.999703168869, 441.80000472068787, 12.200003862380981], [1, 9, 152.09990167617798, 87.59991598129272, 127.90000557899475, 58.60000157356262, 501.49960017204285, 8.700003147125244], [1, 13, 138.5999050140381, 236.29990696907043, 185.59960460662842, 757.4998016357422, 309.5000002384186, 4.100002765655518], [1, 25, 644.8999946117401, 273.5995993614197, 1058.2942011356354, 166.40000748634338, 639.7990009784698, 7.900001764297485], [1, 29, 206.69990396499634, 143.19991064071655, 135.29989385604858, 706.8995971679688, 139.99939966201782, 22.000003814697266], [1, 33, 708.6999976634979, 244.59979820251465, 95.19990062713623, 79.80000162124634, 216.899400472641, 23.20000195503235], [1, 37, 113.69999957084656, 539.099202632904, 231.1997001171112, 5177.492004394531, 72.60000109672546, 30.40001130104065], [1, 42, 216.89959478378296, 109.80009436607361, 212.89979791641235, 550.4985010623932, 308.4997992515564, 4.2000017166137695]

[[1, 5, 202.19930005073547, 87.20011639595032, 273.59890389442444, 621.999703168869, 441.80000472068787, 12.200003862380981], [1, 9, 152.09990167617798, 87.59991598129272, 127.90000557899475, 58.60000157356262, 501.49960017204285, 8.700003147125244], [1, 13, 138.5999050140381, 236.29990696907043, 185.59960460662842, 757.4998016357422, 309.5000002384186, 4.100002765655518], [1, 25, 644.8999946117401, 273.5995993614197, 1058.2942011356354, 166.40000748634338, 639.7990009784698, 7.900001764297485], [1, 29, 206.69990396499634, 143.19991064071655, 135.29989385604858, 706.8995971679688, 139.99939966201782, 22.000003814697266], [1, 33, 708.6999976634979, 244.59979820251465, 95.19990062713623, 79.80000162124634, 216.899400472641, 23.20000195503235], [1, 37, 113.69999957084656, 539.099202632904, 231.1997001171112, 5177.492004394531, 72.60000109672546, 30.40001130104065], [1, 42, 216.89959478378296, 109.80009436607361, 212.89979791641235, 550.4985010623932, 308.4997992515564, 4.2000017166137695]

[[1, 5, 202.19930005073547, 87.20011639595032, 273.59890389442444, 621.999703168869, 441.80000472068787, 12.200003862380981], [1, 9, 152.09990167617798, 87.59991598129272, 127.90000557899475, 58.60000157356262, 501.49960017204285, 8.700003147125244], [1, 13, 138.5999050140381, 236.29990696907043, 185.59960460662842, 757.4998016357422, 309.5000002384186, 4.100002765655518], [1, 25, 644.8999946117401, 273.5995993614197, 1058.2942011356354, 166.40000748634338, 639.7990009784698, 7.900001764297485], [1, 29, 206.69990396499634, 143.19991064071655, 135.29989385604858, 706.8995971679688, 139.99939966201782, 22.000003814697266], [1, 33, 708.6999976634979, 244.59979820251465, 95.19990062713623, 79.80000162124634, 216.899400472641, 23.20000195503235], [1, 37, 113.69999957084656, 539.099202632904, 231.1997001171112, 5177.492004394531, 72.60000109672546, 30.40001130104065], [1, 42, 216.89959478378296, 109.80009436607361, 212.89979791641235, 550.4985010623932, 308.4997992515564, 4.2000017166137695]

[[1, 5, 202.19930005073547, 87.20011639595032, 273.59890389442444, 621.999703168869, 441.80000472068787, 12.200003862380981], [1, 9, 152.09990167617798, 87.59991598129272, 127.90000557899475, 58.60000157356262, 501.49960017204285, 8.700003147125244], [1, 13, 138.5999050140381, 236.29990696907043, 185.59960460662842, 757.4998016357422, 309.5000002384186, 4.100002765655518], [1, 25, 644.8999946117401, 273.5995993614197, 1058.2942011356354, 166.40000748634338, 639.7990009784698, 7.900001764297485], [1, 29, 206.69990396499634, 143.19991064071655, 135.29989385604858, 706.8995971679688, 139.99939966201782, 22.000003814697266], [1, 33, 708.6999976634979, 244.59979820251465, 95.19990062713623, 79.80000162124634, 216.899400472641, 23.20000195503235], [1, 37, 113.69999957084656, 539.099202632904, 231.1997001171112, 5177.492004394531, 72.60000109672546, 30.40001130104065], [1, 42, 216.89959478378296, 109.80009436607361, 212.89979791641235, 550.4985010623932, 308.4997992515564, 4.2000017166137695]

KeyboardInterrupt: 

In [4]:
x = np.array([1, 2, 3, 4, 5])
y = np.array(np.nan)
distance, _ = fastdtw(x, y, dist=euclidean)
print(distance)

TypeError: len() of unsized object

In [63]:
final_df = pd.read_csv('../data/taiko_tables/final.csv')
final_df['dtw_l2'] = np.sqrt(final_df['dtw_0'] ** 2 + final_df['dtw_1'] ** 2 + final_df['dtw_2'] ** 2 + final_df['dtw_5'] ** 2)
tmp_record_df = record_df[['song_id', 'drummer_name']]

final_df = final_df.merge(tmp_record_df, left_on='a_id', right_index=True)
final_df.drop('song_id', inplace=True, axis=1)

final_df = final_df.merge(tmp_record_df, left_on='b_id', right_index=True)
final_df.drop('song_id', inplace=True, axis=1)

final_df = final_df[final_df['dtw_l2'] < 300]
final_df.sort_index(inplace=True)
final_df

,a_id,b_id,dtw_0,dtw_1,dtw_2,dtw_3,dtw_4,dtw_5,dtw_l2,drummer_name_x,drummer_name_y
1,1,9,152.0999,87.5999,127.9000,58.6000,501.4996,8.7000,217.352759,aaaaa,aaaaa
4,1,29,206.6999,143.1999,135.2999,706.8996,139.9994,22.0000,286.393301,aaaaa,carolyn
46,1,214,105.6996,228.4001,120.6999,495.5992,868.4984,4.0000,279.147769,aaaaa,sheep
47,1,218,180.3999,91.2998,179.2995,353.1996,875.8015,6.2000,270.308209,aaaaa,sheep
155,9,13,161.0003,151.5996,131.4998,1006.4997,316.6999,10.0000,257.479577,aaaaa,aaaaa
157,9,29,168.2002,97.1000,63.9998,960.1996,660.5986,26.2000,206.160451,aaaaa,carolyn
199,9,214,142.9008,172.9998,96.2999,630.2992,990.8997,5.9000,244.249975,aaaaa,sheep
200,9,218,210.2005,105.2996,137.9996,411.7996,858.0999,9.7000,272.782396,aaaaa,sheep
201,9,222,223.6009,91.8000,127.8999,152.1999,223.2000,31.6000,275.284484,aaaaa,sheep
228,9,345,145.9003,88.4000,115.4000,456.2000,1238.8986,30.2000,208.160173,aaaaa,kdchang


In [23]:
record_df

,drummer_name,song_id,left_sensor_datetime,right_sensor_datetime,capture_datetime
1,aaaaa,1,L_2018-09-28_111304.csv,R_2018-09-28_111305.csv,capture_2018_09_28_11_13_05
2,aaaaa,2,L_2018-09-28_111610.csv,R_2018-09-28_111611.csv,capture_2018_09_28_11_16_11
3,aaaaa,3,L_2018-09-28_111936.csv,R_2018-09-28_111937.csv,capture_2018_09_28_11_19_37
4,aaaaa,4,L_2018-09-28_112353.csv,R_2018-09-28_112355.csv,capture_2018_09_28_11_23_55
5,aaaaa,1,L_2018-09-28_112912.csv,R_2018-09-28_112913.csv,capture_2018_09_28_11_29_13
6,aaaaa,2,L_2018-09-28_113242.csv,R_2018-09-28_113244.csv,capture_2018_09_28_11_32_44
7,aaaaa,3,L_2018-09-28_113531.csv,R_2018-09-28_113532.csv,capture_2018_09_28_11_35_32
8,aaaaa,4,L_2018-09-28_113909.csv,R_2018-09-28_113910.csv,capture_2018_09_28_11_39_10
9,aaaaa,1,L_2018-09-28_114340.csv,R_2018-09-28_114340.csv,capture_2018_09_28_11_43_41
10,aaaaa,2,L_2018-09-28_114710.csv,R_2018-09-28_114713.csv,capture_2018_09_28_11_47_13


In [26]:
play_result_df = pd.read_csv('../data/taiko_tables/taiko_play_result.csv')
play_result_df = play_result_df[['capture_datetime', 'bad']]
rec_df = record_df.merge(play_result_df, how='left', on='capture_datetime')
# rec_df.dropna(inplace=True)
# rec_df['bad'] = rec_df['bad'].astype(np.int32)
# rec_df = rec_df[rec_df['song_id'] == 1]
rec_df

,drummer_name,song_id,left_sensor_datetime,right_sensor_datetime,capture_datetime,bad
0,aaaaa,1,L_2018-09-28_111304.csv,R_2018-09-28_111305.csv,capture_2018_09_28_11_13_05,0.0
1,aaaaa,2,L_2018-09-28_111610.csv,R_2018-09-28_111611.csv,capture_2018_09_28_11_16_11,0.0
2,aaaaa,3,L_2018-09-28_111936.csv,R_2018-09-28_111937.csv,capture_2018_09_28_11_19_37,0.0
3,aaaaa,4,L_2018-09-28_112353.csv,R_2018-09-28_112355.csv,capture_2018_09_28_11_23_55,0.0
4,aaaaa,1,L_2018-09-28_112912.csv,R_2018-09-28_112913.csv,capture_2018_09_28_11_29_13,0.0
5,aaaaa,2,L_2018-09-28_113242.csv,R_2018-09-28_113244.csv,capture_2018_09_28_11_32_44,0.0
6,aaaaa,3,L_2018-09-28_113531.csv,R_2018-09-28_113532.csv,capture_2018_09_28_11_35_32,0.0
7,aaaaa,4,L_2018-09-28_113909.csv,R_2018-09-28_113910.csv,capture_2018_09_28_11_39_10,1.0
8,aaaaa,1,L_2018-09-28_114340.csv,R_2018-09-28_114340.csv,capture_2018_09_28_11_43_41,0.0
9,aaaaa,2,L_2018-09-28_114710.csv,R_2018-09-28_114713.csv,capture_2018_09_28_11_47_13,0.0
